In [93]:
import pandas as pd

In [95]:
#load topics and use definitions from 
#https://docs.google.com/spreadsheets/d/1tjx8B9T5_ce4DBCKBwPoCzLMsg7jafXLGkzmGYLlJPM/edit#gid=0

filename = 'id+topic_scores.pd'
topics = pd.read_pickle(filename)

topic_names = {
1: 'metaphysics',
2: '',
3: 'abortions',
4: 'geopolitcs',
5: 'racism',
6: 'healthcare',
7: 'psychology/violence',
8: 'christianity',
9: 'gender',
10: '(Radical) Islam',
11: 'movies',
12: 'music',
13: 'libertarianism',
14: '',
15: '',
16: 'american history',
17: 'canada',
18: 'ukranian conflict',
19: 'US elections & Trump',
20: 'South African white farmer genocide',
21: 'Academia',
22: 'European migration crisis',
23: '',
24: 'Gaming',
25: '',
26: '',
27: '',
28: '(Social) media',
29: 'U.S. party politics',
30: 'mood and emotions',
31: 'eugenics',
32: '',
33: 'Fitness & Diet',
34: 'finance (government and tax)',
35: '',
36: 'legal discourse',
37: 'police violence',
38: 'Jordan Peterson/Sam Harris debate',
39: 'economics',
40: '',
41: 'UK politics',
42: 'Conspiracy / Occult',
43: 'Socialism / Communism / left totalitarianism',
44: '(anti-)semitism',
45: 'climate change',
46: '',
47: 'Islam',
48: 'European history',
49: 'Brexit',
50: 'Education',
}

topics['topic_name'] = topics['topic'].map(topic_names, na_action=None)
topics['topic_score'] = topics['topic_score'].astype('float')

#picking main topic per video to be able to look at views, I don't want any duplicate videos
idx = topics.groupby(['id'])['topic_score'].transform(max) == topics['topic_score']
topics = topics[idx]

topics.head()


,id,topic,topic_score,topic_name
0,s_H4Ndtr_Ik,9,0.20098,gender
5,0pXY5UwGK5M,44,0.27390,(anti-)semitism
7,dSmEtfQoSwU,22,0.28512,European migration crisis
9,7a2wKoDqbjw,2,0.51453,NOT RELEVANT
12,u28lc7Y3bnE,2,0.36274,NOT RELEVANT


In [96]:
list(topics)

['id', 'topic', 'topic_score', 'topic_name']

In [97]:
topics.shape

(54072, 4)

In [98]:
#get metadata from captions from https://drive.google.com/open?id=13f2fYPIsiednDBTMhd7rvCyikD_R6405

filename = 'captions_metadata.csv'
metadata = pd.read_csv(filename, 
                       usecols=['id', 'date', 'title', 'channel'])

list(metadata)

['id', 'date', 'title', 'channel']

In [99]:
metadata.head()


,id,date,title,channel
0,--2xbXFwBZc,2017-11-04T20:00:01.000Z,THE NOVEMBER 4TH ANTIFA APOCALYPSE,Tim Pool
1,--31D-c_jaQ,2009-05-11T22:11:17.000Z,UKIP Nigel Farage on Radio 5 Live with Nicky C...,voteleavemedia
2,--8EWeHr2Os,2017-07-13T23:19:26.000Z,Warning for Canada: 128M Europeans in energy p...,Rebel Canada
3,--MWow7i0RU,2018-02-26T19:08:32.000Z,Gov. Matt Bevin: Reforming foster care systems...,American Enterprise Institute
4,--N00nkKZuQ,2018-04-04T14:30:00.000Z,Wie aus einem Verkehrsprojekt eine Müllhalde w...,AfD Fraktion Abgeordnetenhaus Berlin


In [100]:
metadata.shape

(57243, 4)

In [112]:
#get actual metadata from https://drive.google.com/open?id=13f2fYPIsiednDBTMhd7rvCyikD_R6405
# this data is not available here yet. Dimitri has it, he should upload it! :D

filename = 'vids_metadata.csv' 
real_metadata = pd.read_csv(filename, low_memory=False)

real_metadata.rename(index=str, columns={"videoId": "id"},inplace=True)
real_metadata.fillna(0,inplace=True)

real_metadata['views'] = real_metadata['views'].astype('int')
real_metadata['likes'] = real_metadata['likes'].astype('int')
real_metadata['dislikes'] = real_metadata['dislikes'].astype('int')
real_metadata['comments'] = real_metadata['comments'].astype('int')

list(real_metadata)


['Unnamed: 0',
 'id',
 'description',
 'tags',
 'views',
 'likes',
 'dislikes',
 'comments']

In [113]:
real_metadata.head()


,Unnamed: 0,id,description,tags,views,likes,dislikes,comments
0,0,UxN5JIqU_8w,Want to watch the full show every day? Join #M...,"['steven crowder', 'stephen crowder', 'louder ...",142510,5910,142,2655
1,1,ur1bjqQ5ceA,Steven Crowder takes Change My Mind to the Whi...,"['steven crowder', 'stephen crowder', 'louder ...",1592958,71902,1740,26667
2,2,P5tNUJUykMc,"Tune in Wednesday, August 29th 2018 to see the...","['steven crowder', 'stephen crowder', 'louder ...",276836,15616,173,2874
3,3,2oa5SMjYgUU,It’s a full house tonight! We host Lauren Sout...,"['steven crowder', 'stephen crowder', 'louder ...",256090,9717,245,3608
4,4,C1chmEfIgb0,Tonight we’re joined in studio by Lauren South...,"['steven crowder', 'stephen crowder', 'louder ...",499764,25461,498,5904


In [103]:
real_metadata.shape

(92081, 8)

In [114]:
#merge all data and fix date 

df = metadata.merge(topics,on='id',how='left').merge(real_metadata,on='id',how='left')
df.sort_values(by=['views'],ascending=False,inplace=True)

df['date'] = pd.to_datetime(df['date'])
df['YearMonth'] = df['date'].map(lambda x: 100*x.year + x.month)
df['Year'] = df['date'].map(lambda x: x.year)

list(df)
df.sample(n=100)

,id,date,title,channel,topic,topic_score,topic_name,Unnamed: 0,description,tags,views,likes,dislikes,comments,YearMonth,Year
20300,J63WG0ihRyk,2017-10-03 16:22:56,Stop Believing Everything You Think,Cy Wakeman,41.0,0.28268,UK politics,55564,In my conversation with Gary Vaynerchuk on The...,"['leadership', 'leadership development', 'mind...",1003,4,0,0,201710,2017
850,-pyYoL9ngtE,2014-05-16 10:59:34,Nigel Farage v James O'Brien: Live On LBC,LBC,41.0,0.29005,UK politics,22430,Nigel Farage has challenged James O'Brien to a...,"['Nigel Farage (TV Actor)', ""james o'brien"", '...",778201,5078,948,5605,201405,2014
36364,ZU-gcvMtJaw,2012-04-25 16:26:03,Did God Create Multiple Universes,ReasonableFaithOrg,1.0,0.88908,metaphysics,2556,"Robert Lawrence Kuhn (host of PBS' ""Closer to ...","['Multiverse', 'Multiple', 'Universe', 'Parall...",1730,22,3,0,201204,2012
55833,sBm8i96ZGcQ,2016-09-03 17:57:39,Why Spanking Does Not Work | Elizabeth Gershof...,Stefan Molyneux,9.0,0.25160,gender,2705,MP3: http://www.fdrpodcasts.com/#/3400/why-spa...,"['state', 'god', 'family', 'atheism', 'atheist...",44723,1320,412,1739,201609,2016
5588,4ZvbWht0PDg,2015-01-14 20:24:20,Interrupties inbreng Geert Wilders bij debat o...,PVVpers,2.0,0.59394,NOT RELEVANT,20799,Interrupties inbreng Geert Wilders bij debat o...,"['Geert Wilders (Politician)', 'Politics (TV G...",196708,380,135,8,201501,2015
14638,DUGaXY4uHRs,2011-10-22 15:47:08,"Nigel Farage despre aderarea Romaniei la UE: ""...",În Linie Dreaptă,49.0,0.32713,Brexit,47208,http://inliniedreapta.net/\n\nUrmariti-ne pe F...,"['nigel farage', 'romania']",2573,18,5,21,201110,2011
42363,eeynAX57ix0,2018-05-13 04:22:17,Dawson v. Fuentes | TPS #15,Jean-Francois Gariépy,1.0,0.14786,metaphysics,21081,A debate on religion and race between Ryan Daw...,"['youtube', 'warski', 'stream', 'news', 'relig...",35953,1998,84,1230,201805,2018
15033,DtUbiX-MO7Q,2017-02-17 03:41:52,Discussion: J. R. R. Tolkien and Old English,Leornende Eald Englisc,48.0,0.45673,European history,41682,Previous Video:\nhttps://www.youtube.com/watch...,"['Tolkien', 'Lord Of the Rings', 'Old English'...",2154,155,4,48,201702,2017
35212,YGXWLYcKXGo,2017-12-19 17:00:06,How To Increase Dopamine Levels Naturally,Philip G,33.0,0.59947,Fitness & Diet,23738,The Ripped Dude Program ➡ https://goo.gl/U43P5...,"['how to increase dopamine levels', 'benefits ...",3821,184,9,34,201712,2017
24368,NFIHovqHqH8,2010-07-20 15:00:07,Beogradski Sindikat - Za sve moje ljude (2010),sundjer bob,2.0,0.66555,NOT RELEVANT,53268,0,"['Beogradski', 'Sindikat', 'Za', 'sve', 'moje'...",40855,104,3,20,201007,2010


In [118]:
#aggregate and push to csv

topics_per_month = df.groupby(['Year', 'topic','topic_name'])['views'].agg(['sum','count']).reset_index()
topics_per_month['sum'] = topics_per_month['sum'].astype('int')

topics_per_month.to_csv('topics_over_time.csv')
topics_per_month

,Year,topic,topic_name,sum,count
0,2006,1.0,metaphysics,35146,1
1,2006,4.0,geopolitcs,40018,3
2,2006,5.0,racism,24172,1
3,2006,8.0,christianity,587933,1
4,2006,12.0,music,551874,6
5,2006,15.0,,813541,2
6,2006,16.0,american history,3470,1
7,2006,20.0,South African white farmer genocide,132191,1
8,2006,23.0,,464377,1
9,2006,27.0,,44318,2
